In [5]:
from analysis.ipstartup import *
import aws
import docker
import fabric.api as fab
import fabric

fab.output['everything'] = True
fab.env.host_string = aws.ec2.Instance(aws.getId("sm0")).public_ip_address

[root:INFO]:starting (cellevents.py\31, time=20:29)


time: 1.13 s


In [ ]:
aws.create_server(disksize=15)

In [6]:
aws.getInstances()

[root:INFO]:starting (cellevents.py\31, time=20:29)


,name,instance_id,image,type,state,ip
0,sm0,i-054a9985c431ecbed,ami-c51e3eb6,t2.micro,running,34.248.84.101
1,sm1,i-0933f9f9fabdcdc28,ami-c51e3eb6,t2.micro,terminated,None


time: 422 ms


In [8]:
docker.install_base()

[root:INFO]:starting (cellevents.py\31, time=19:57)


TypeError: connect() missing 1 required positional argument: 'key'

time: 11 ms


In [7]:
fab.run("docker ps -a")

[root:INFO]:starting (cellevents.py\31, time=20:29)


[34.248.84.101] run: docker ps -a
[34.248.84.101] out: CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
[34.248.84.101] out: c3a796b9de0a        kaggle/python       "/usr/bin/tini -- /bi"   About an hour ago   Up About an hour    0.0.0.0:8888->8888/tcp   notebook
[34.248.84.101] out: 



'CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES\r\nc3a796b9de0a        kaggle/python       "/usr/bin/tini -- /bi"   About an hour ago   Up About an hour    0.0.0.0:8888->8888/tcp   notebook'

time: 683 ms


In [8]:
docker.gettasks()

[root:INFO]:starting (cellevents.py\31, time=20:29)


[34.248.84.101] run: docker inspect --format='{{.Name}}' $(docker ps -q)
[34.248.84.101] out: /notebook
[34.248.84.101] out: 

[34.248.84.101] run: docker exec notebook ps -eo args | grep python || true
[34.248.84.101] out: /opt/conda/bin/python /opt/conda/bin/jupyter-notebook
[34.248.84.101] out: 



,container,task
0,notebook,/opt/conda/bin/python /opt/conda/bin/jupyter-n...


time: 359 ms


In [ ]:
i1 = aws.create_server(ami["ImageId"], servertype="gpu", disksize=15)

In [9]:
docker.install_projects()

[root:INFO]:starting (cellevents.py\31, time=20:30)


[34.248.84.101] run: mkdir basics || true
[34.248.84.101] out: mkdir: cannot create directory ‘basics’: File exists
[34.248.84.101] out: 

[localhost] local: git archive HEAD > basics.tar
[34.248.84.101] put: C:\Users\s\Documents\py\apps\aws\..\basics\basics.tar -> /home/ec2-user/basics.tar
[34.248.84.101] run: tar xf basics.tar -C basics
[34.248.84.101] run: rm basics.tar
[34.248.84.101] run: mkdir analysis || true
[34.248.84.101] out: mkdir: cannot create directory ‘analysis’: File exists
[34.248.84.101] out: 

[localhost] local: git archive HEAD > analysis.tar
[34.248.84.101] put: C:\Users\s\Documents\py\apps\aws\..\analysis\analysis.tar -> /home/ec2-user/analysis.tar
[34.248.84.101] run: tar xf analysis.tar -C analysis
[34.248.84.101] run: rm analysis.tar
[34.248.84.101] run: mkdir meetup || true
[34.248.84.101] out: mkdir: cannot create directory ‘meetup’: File exists
[34.248.84.101] out: 

[localhost] local: git archive HEAD > meetup.tar
[34.248.84.101] put: C:\Users\s\Documents\

In [ ]:
docker.restart_notebook()

In [10]:
docker.restart_meetup()

[root:INFO]:starting (cellevents.py\31, time=20:30)


[34.248.84.101] run: docker rm -f meetup || true
[34.248.84.101] out: Error response from daemon: No such container: meetup
[34.248.84.101] out: 

[34.248.84.101] run: docker run -v $PWD:/v1 -w=/v1 -d -i --name meetup kaggle/python
[34.248.84.101] out: fa83da905db2f15fc86c1357611f884d875b1f8cc16cca59623142f6a43b1b3b
[34.248.84.101] out: 

[34.248.84.101] run: docker exec meetup python basics/pathconfig.py
[34.248.84.101] out: [root:INFO]:starting (pathconfig.py\10, time=20:30)
[34.248.84.101] out: 

[34.248.84.101] run: docker exec -d meetup python meetup/meetup.py
time: 2.12 s


In [11]:
d(docker.gettasks("python"))

[root:INFO]:starting (cellevents.py\31, time=20:30)


[34.248.84.101] run: docker inspect --format='{{.Name}}' $(docker ps -q)
[34.248.84.101] out: /meetup
[34.248.84.101] out: /notebook
[34.248.84.101] out: 

[34.248.84.101] run: docker exec meetup ps -eo args | grep python || true
[34.248.84.101] out: python meetup/meetup.py
[34.248.84.101] out: 

[34.248.84.101] run: docker exec notebook ps -eo args | grep python || true
[34.248.84.101] out: /opt/conda/bin/python /opt/conda/bin/jupyter-notebook
[34.248.84.101] out: 



,container,task
0,meetup,python meetup/meetup.py
1,notebook,/opt/conda/bin/python /opt/conda/bin/jupyter-n...


time: 575 ms


In [ ]:
import boto3
s3=boto3.resource("s3")
s3.create_bucket(Bucket="simonm3")
for a in s3.buckets.all():
    print(a)

In [ ]:
for a in s3.Bucket('simonm3').objects.all():
    print(a.bucket_name, a.key)

In [ ]:
s3a = s3.Bucket('simonm3')
s3a.put_object(Key='test2.txt', Body="just testing2")
#s3a.download_file(Key="test.txt", Filename="temp")